# Анализ информации из базы данных о сервисе чтения книг  

**Заказчик:** Компания, купившая крупный сервис для чтения книг по подписке.

**Задача:** Проанализировать базу данных содержащую информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Описание данных:** <br/>
`Таблица books`
```
Содержит данные о книгах:
•	book_id — идентификатор книги;
•	author_id — идентификатор автора;
•	title — название книги;
•	num_pages — количество страниц;
•	publication_date — дата публикации книги;
•	publisher_id — идентификатор издателя.
```
`Таблица authors`
```
Содержит данные об авторах:
•	author_id — идентификатор автора;
•	author — имя автора.
```
`Таблица publishers`
```
Содержит данные об издательствах:
•	publisher_id — идентификатор издательства;
•	publisher — название издательства;
```
`Таблица ratings`
```
Содержит данные о пользовательских оценках книг:
•	rating_id — идентификатор оценки;
•	book_id — идентификатор книги;
•	username — имя пользователя, оставившего оценку;
•	rating — оценка книги.
```
`Таблица reviews`
```
Содержит данные о пользовательских обзорах:
•	review_id — идентификатор обзора;
•	book_id — идентификатор книги;
•	username — имя автора обзора;
•	text — текст обзора.
```

**Схема данных**
<img src="./shema.png">

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

In [2]:
# Выясним какие ещё таблицы доступны в базе
display(pd.io.sql.read_sql('''

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
      schemaname != 'information_schema';

''', con = engine))


# смотрим на типы столбцов в интересующих таблицах
display(pd.io.sql.read_sql('''
SELECT 
    table_name, 
    column_name, 
    data_type, 
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
''', con = engine))

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,orders,praktikum_admin,None,True,False,False,False
1,public,visits,praktikum_admin,None,True,False,False,False
2,public,advertisment_costs,praktikum_admin,None,True,False,False,False
3,public,authors,praktikum_admin,None,True,False,True,False
4,public,second,praktikum_student,None,False,False,False,False
5,public,second_b,praktikum_student,None,False,False,False,False
6,public,publishers,praktikum_admin,None,True,False,True,False
7,public,author,praktikum_student,None,True,False,False,False
8,public,reviews,praktikum_admin,None,True,False,True,False
9,public,ratings,praktikum_admin,None,True,False,True,False


,table_name,column_name,data_type,is_nullable
0,authors,author_id,integer,NO
1,authors,author,text,YES
2,publishers,publisher_id,integer,NO
3,publishers,publisher,text,YES
4,reviews,review_id,integer,NO
5,reviews,book_id,integer,YES
6,reviews,username,text,YES
7,reviews,text,text,YES
8,ratings,rating_id,integer,NO
9,ratings,book_id,integer,YES


Посчитаем, сколько книг вышло после 1 января 2000 года

In [3]:
query = '''SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '2000-01-01'            
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Для каждой книги посчитаем количество обзоров и среднюю оценку

In [4]:

query = '''WITH i AS
             (SELECT  b.book_id book_id,
                    b.title title,
                    AVG(ra.rating) avg_rating
            FROM books as b
            LEFT JOIN ratings ra ON b.book_id=ra.book_id
            GROUP BY b.book_id, b.title) 
           
           SELECT i.book_id,
                i.title,
                i.avg_rating,
                COUNT(re.review_id) rew_cnt
           FROM i
           LEFT JOIN reviews re ON i.book_id=re.book_id
           GROUP BY 1,2,3
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,avg_rating,rew_cnt
0,200,Dreamcatcher,2.800000,3
1,593,Sputnik Sweetheart,3.666667,3
2,735,The God Delusion,4.000000,4
3,570,Silent Spring,3.000000,2
4,604,Surely You're Joking Mr. Feynman!: Adventures...,4.250000,3
...,...,...,...,...
995,70,American Gods (American Gods #1),4.000000,5
996,488,Of Mice and Men,3.622951,5
997,886,The Tombs of Atuan (Earthsea Cycle #2),4.666667,3
998,881,The Tenth Circle,3.500000,4


Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [5]:
query = '''WITH i AS
                (SELECT p.publisher_id publ_id,
                        p.publisher publ,
                 COUNT(b.book_id) book_cnt
                 FROM books b
                 JOIN publishers p ON b.publisher_id=p.publisher_id
                 WHERE b.num_pages > 50
                 GROUP BY p.publisher_id, p.publisher)
          SELECT i.publ_id,
                i.publ,
                i.book_cnt
          FROM i
          WHERE i.book_cnt in (SELECT MAX(i.book_cnt)
                                FROM i)   
                        
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,publ_id,publ,book_cnt
0,212,Penguin Books,42


Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [6]:
query = '''
                     
        WITH i AS (SELECT a.author_id author_id,
                           a.author author,
                           AVG(ra.rating) avg_rating
                    FROM authors a
                    LEFT JOIN books b ON a.author_id=b.author_id
                    LEFT JOIN ratings ra ON b.book_id=ra.book_id
                    WHERE b.book_id IN (SELECT j.book_id
                                        FROM (SELECT b.book_id book_id,
                                                      COUNT(ra.rating_id)
                                               FROM books b
                                               LEFT JOIN ratings ra ON b.book_id=ra.book_id
                                               GROUP BY b.book_id
                                               HAVING COUNT(ra.rating_id)>=50) AS j)
              
                    GROUP BY 1,2)
        
        SELECT i.author_id,
               i.author,
               i.avg_rating
        FROM i
        WHERE i.avg_rating IN (SELECT MAX(i.avg_rating)
                                FROM i)
          
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [7]:
query = '''SELECT AVG(i.rev_cnt)
        FROM
                (SELECT  username,
                        COUNT(review_id) rev_cnt
                FROM reviews
                WHERE username IN (SELECT username
                                        FROM ratings
                                        GROUP BY 1
                                        HAVING COUNT(rating_id)>48) 
                GROUP BY 1) AS i
       '''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


# Выводы

* после 1 января 2000 года вышло 819 книг 
* для каждой книги получен средний рейтинг и количество обзоров
* издательство, которое выпустило наибольшее число книг (42 шт.) толще 50 страниц - Penguin Books
* автор с самой высокой средней оценкой книг (4.3) среди тех что имеют 50 и более оценок: J.K. Rowling/Mary GrandPré	
* среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24